In [1]:
from absbox import API, mkDeal,EnginePath,readInspect
import pandas as pd 
#localAPI = API("http://localhost:8081/", lang="english",check=False)
name = "Deal"
dates = {
    "cutoff":"2024-10-31",
    "closing":"2024-10-31",
    "firstPay":"2024-11-30",
    "stated": "2027-10-31",
    "poolFreq": "MonthEnd",
    "payFreq": "MonthEnd"
}

amortising_loan_monthly = ["Mortgage"
            ,{"originBalance": 6922.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 12
            ,"freq": "Monthly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 6922.22
            ,"currentRate": 0.16
            ,"remainTerm": 12
            ,"status": "current"}]
amortising_loan_quarterly = ["Mortgage"
            ,{"originBalance":  2_966.67    
            ,"originRate": ["fix",0.16]
            ,"originTerm": 4
            ,"freq": "Quarterly"
            ,"type": "Even"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 2_966.67
            ,"currentRate": 0.16
            ,"remainTerm": 4
            ,"status": "current"}]

bullet_loans = ["Loan"
            ,{"originBalance": 12_222.22
            ,"originRate": ["fix",0.16]
            ,"originTerm": 6
            ,"freq": "Monthly"
            ,"type": "i_p"
            ,"originDate": "2024-10-31"}
            ,{"currentBalance": 12_222.22
            ,"currentRate": 0.16
            ,"remainTerm": 6
            ,"status": "current"}]

j = {"pool_amortizing": {'assets':[amortising_loan_monthly]*100 + [amortising_loan_quarterly]*100},"pool_bullet": {'assets':[bullet_loans]*10}}


accounts = {
    "collections_interest": {"balance": 0},
    "collections_principal_amortizing": {"balance": 0},
    "collections_principal_bullet": {"balance": 0},
    "revolBuyAcc1": {"balance": 0},
    "revolBuyAcc2": {"balance": 0},
    "dealAcc": {"balance": 0},
}
bonds = {
    "A1": {
        "balance": 1_000_000.00,
        "rate": 0.114,
        "originBalance": 30_000_000,
        "originRate": 0.114,
        "startDate": "2024-10-31",
        "rateType": {"floater":[0.049, "SOFR1M",0.065,"MonthEnd"]},
        "maturityDate": "2027-10-31",
        "bondType": {"Sequential":None},
    },
    "EQ": {
        "balance":  111_111.00,
        "rate": 0.0,
        "originBalance": 111_111.00,
        "originRate": 0.0,
        "startDate": "2024-10-31",
        "maturityDate": "2027-10-31",
        "rateType": {"Fixed": 0.00},
        "bondType": {"Equity": None},
    },
}

                                

fees = {"utilisation_fee": {"type": {"annualPctFee": [("excess",("originalBondBalance", "A1"),("bondBalance", "A1"),),0.007],},
                            "feeStart": "2025-01-31","feeEnd": "2026-10-31"},
        "technology_fee": {"type": {"annualPctFee": [("poolBalance",),0.005],},
                            "feeStart": "2025-01-31"},}
waterfall = {
                    "closingDay":[
                        ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  1_780_000.00)}], 
                        ["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 220_000.00)}],
                        ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  197_777.78)}],
                        ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  24_444.44)}],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                        ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],

                    ],
                    "Accelerated":[
                                ["calcInt", "A1"],
                                ["fundWith", "revolBuyAcc1", "A1",{"formula": ("const",  2_670_000.00+330_000)}], 
                                #["fundWith", "revolBuyAcc2", "A1",{"formula": ("const", 330_000.00)}],
                                ["fundWith", "revolBuyAcc1", "EQ",{"formula": ("const",  296_666.67)}],
                                ["fundWith", "revolBuyAcc2", "EQ",{"formula": ("const",  36_666.67)}],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.89,),),
                                    "support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "revolving":[
                                ["calcInt", "A1"],
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ['transfer','collections_principal_amortizing','revolBuyAcc1'],
                                ['transfer','collections_principal_bullet','revolBuyAcc2'],
                                ["transfer","collections_interest","revolBuyAcc1",{
                                "formula": ("-",("bondBalance","A1"),
                                            ("factor",
                                             ("sum",("poolBalance","pool_amortizing"),
                                                    ("poolBalance","pool_bullet"),
                                              ("accountBalance","revolBuyAcc1","revolBuyAcc2","collections_principal_amortizing","collections_principal_bullet"),),
                                             0.9,),)
                                    ,"support": ["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]},
                                    ],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc1",None,"Pool1","pool_amortizing"],
                                ["buyAsset2",["Current|Defaulted", 1.0, 0],"revolBuyAcc2",None,"Pool2","pool_bullet"],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],
                    "amortizing":[
                                ["calcInt", "A1"],  # actions if deal is in Amortizing status
                                ["calcAndPayFee","collections_interest",[ "utilisation_fee","technology_fee"],
                                 {"support":["support",["account","collections_principal_amortizing"],["account","collections_principal_bullet"]]}],
                                ['transfer','collections_principal_amortizing','dealAcc'],
                                ['transfer','collections_principal_bullet','dealAcc'],
                                ["accrueAndPayInt", "collections_interest", ["A1"]],
                                ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                                ["payIntResidual", "collections_interest", "EQ"],
                                ],

                    "cleanUp": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payPrin", "dealAcc", ["EQ"]],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ],

                    "Defaulted": [
                            ['transfer','collections_principal_amortizing','dealAcc'],
                            ['transfer','collections_principal_bullet','dealAcc'],
                            ["accrueAndPayInt", "collections_interest", ["A1"]],
                            ['transfer','collections_interest','dealAcc'],
                            ["payPrin","dealAcc",["A1"],{"support":["account","collections_interest"]}],
                            ["payIntResidual", "dealAcc", "EQ"],
                            ["payPrin", "dealAcc", ["EQ"]],],
                            
                }

collects = [
            [["pool_amortizing"],"CollectedPrincipal", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedRecoveries", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedPrepayment", "collections_principal_amortizing"],
            [["pool_amortizing"],"CollectedInterest", "collections_interest"],
            [["pool_bullet"],"CollectedPrincipal", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedRecoveries", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedPrepayment", "collections_principal_bullet"],
            [["pool_bullet"],"CollectedInterest", "collections_interest"],
        ]
defrate3periods = \
   [ ("ratio",("curPoolCollectionStats", None, _*-1, "Defaults"),("curPoolCollectionStats", None, _*-1, "BegBalance"))
   for _ in range(3) ]

trigger = {
            "AfterCollect": {
                "revolving_trigger": {
                    "condition": ["any", [">", "2026-01-31"]],
                    "effects": ("newStatus", "Revolving"),
                    "status": False,
                    "curable": False,
                },
                "amortization_trigger": {
                    "condition": ["any", [">=", "2026-10-31"]],
                    "effects": ("newStatus", "Amortizing"),
                    "status": False,
                    "curable": False,
                },
                "DefaultRate_trigger": { #Rolling Default rate trigger
            "condition": ["all", 
                          [">=", "2025-01-31"],
                          [("avgRatio",*defrate3periods),">", 0.06],
            ],
            "effects": ("newStatus", "Defaulted"),
            "status": False,
            "curable": False,
        },},
        "AfterDistribution": {
                "LTV_trigger": {
                    "condition": ["all",
                                  [">", "2024-10-31"],
                                  [
                        (
                            "ratio",
                            ("bondBalance","A1"),
                            ("sum",
                             ("poolBalance","pool_amortizing"),
                             ("poolBalance","pool_bullet"),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             ),
                        ),
                            ">",
                        0.9,
                    ],],
                    "effects": ("newStatus", "Defaulted"),
                    "status": False,
                    "curable": True,
                },
            }, 
        }

deal_data = {
            "name": name,
            "dates": dates,
            "pool": j,
            "accounts": accounts,
            "fees": fees,
            "bonds": bonds,
            "waterfall": waterfall,
            "collect": collects,
            "trigger": trigger,
            "status": ("PreClosing","Accelerated"),
        }

deal = mkDeal(deal_data)
revol_asset1 = [amortising_loan_monthly] +[amortising_loan_quarterly]
#revol_asset1 = [amortising_loan_quarterly]
revol_asset2 = bullet_loans

cdr_assump = 0

cdr_assump_1 = 0

revolvingPool1 = (["constant",*revol_asset1]
                 ,("Pool",("Mortgage",{"CDR":cdr_assump_1},None,None,None
                                                                   )
                   ,None
                   ,None))

revolvingPool2 = (["constant",revol_asset2]
                 ,("Pool",("Loan",{"CDR":cdr_assump_1},None,None,None)
                   ,None
                   ,None))

amortisingPoolAssump = ("Pool",("Mortgage",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

bulletPoolAssump = ("Pool",("Loan",{"CDR":cdr_assump},None,None,None)
                   ,None
                   ,None)

localAPI = API(EnginePath.LOCAL, lang="english",check=False)
r = localAPI.run(deal,
                 poolAssump = ("ByPoolId",
                                    {"pool_amortizing":amortisingPoolAssump
                                    ,"pool_bullet":bulletPoolAssump}
                                    ),
         runAssump=[#tuple(trigger_inspect_creation(deal)),
             ("interest", ("SOFR1M", 0.049), ("EURIBOR1M", 0.0858)),
             ("inspect",("MonthEnd",("bondBalance", "A1")),
              ("MonthEnd",("poolBalance","pool_amortizing")),
              ("MonthEnd",("poolBalance","pool_bullet")),
              ("MonthEnd", ("sum",
                             ("poolBalance","pool_amortizing"),
                             ("poolBalance","pool_bullet"),
                             ("accountBalance","revolBuyAcc1","revolBuyAcc2","dealAcc","collections_principal_amortizing","collections_principal_bullet"),
                             )),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Principal")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Interest")),
              ("MonthEnd",("curPoolCollectionStats", None, 0, "Defaults"))
              ,("MonthEnd",("accountBalance","revolBuyAcc1","revolBuyAcc2",))
             ,("MonthEnd",("curPoolCollectionStats", None, 0, "BegBalance"))
                              ),
              ("call", {"afterDate":"2027-10-31"}),
              ("revolving",{"Pool1":revolvingPool1,"Pool2":revolvingPool2})
              #,("stop","2024-12-01")                                
             
         ]
         ,read=True)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.29.20

Warning Message from server:
Bond EQ is not paid off
Account dealAcc has cash to be distributed

In [2]:
r['result']['waterfallInspect']

In [3]:
from absbox import readInspect
readInspect(r['result'])

,"<AccBalance:<revolBuyAcc1>,<revolBuyAcc2>>",<CurrentBondBalanceOf:<A1>>,<CurrentPoolBalance:<PoolName:pool_amortizing>>,<CurrentPoolBalance:<PoolName:pool_bullet>>,"<PoolCollectionStats:<0>,<['CollectedInterest']>,<None>>","<PoolCollectionStats:<0>,<['CollectedPrincipal']>,<None>>","<PoolCollectionStats:<0>,<['CurBegBalance']>,<None>>","<PoolCollectionStats:<0>,<['NewDefaults']>,<None>>","<Sum:<CurrentPoolBalance:<PoolName:pool_amortizing>>,<CurrentPoolBalance:<PoolName:pool_bullet>>,<AccBalance:<revolBuyAcc1>,<revolBuyAcc2>,<dealAcc>,<collections_principal_amortizing>,<collections_principal_bullet>>>"
Date,,,,,,,,,
2024-10-31,0.0,3000000.00,2966666.77,366666.64,0.00,0.00,1111111.20,0.0,3333333.41
2024-11-30,0.0,6000000.00,6267733.54,403333.31,32509.69,173055.32,3333333.41,0.0,6671066.85
2024-12-31,0.0,9000000.00,9569576.66,439999.98,63269.61,375712.35,6671066.85,0.0,10009576.64
2025-01-31,0.0,12000000.00,12907800.60,476666.65,128697.00,812735.80,10009576.64,0.0,13384467.25
2025-02-28,0.0,15000000.00,16250271.76,513333.32,165546.47,1092934.95,13384467.25,0.0,16763605.08
2025-03-31,0.0,18000000.00,19584998.47,549999.99,199329.51,1366924.37,16763605.08,0.0,20134998.46
2025-04-30,0.0,21000000.00,22959226.25,586666.66,261397.40,2265859.71,20134998.46,0.0,23545892.91
2025-05-31,0.0,24000000.00,26333381.55,623333.33,300889.10,2302868.27,23545892.91,0.0,26956714.88
2025-06-30,0.0,27000000.00,29671905.30,660000.00,336877.98,2667114.85,26956714.88,0.0,30331905.30
